In [2]:
%load_ext watermark

import numpy as np
import pandas as pd
import xarray as xr

%watermark -iv

xarray: 2023.10.1
numpy : 1.24.4
sys   : 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
pandas: 2.2.0



# Tree-ring site analysis in CESM2 historical simulation
**Simulation details:**
* Atmospheric forcing data: GSWP3v1
* Spatial resolution: 1.25deg x 1deg
* Temporal resolution: month
* CLM version: CLM5, CTSM5.1
* CESM version: cesm2_3_alpha02c

**Here's an example for stomatal conductance (`GSSUNLN`):**

In [43]:
maindir = '/glade/work/bbuchovecky/WUE_analysis/sites_from_gridded'

# Load indices for sites with angiosperm (broadleaf) PFTs
sites_with_angtreepfts = xr.open_dataset(f'{maindir}/sites_with_angtreepfts.nc')
sites_with_angtreepfts = sites_with_angtreepfts['sites_with_angtreepfts']

# Load indices for sites with gymnosperm (needleleaf) PFTs
sites_with_gymtreepfts = xr.open_dataset(f'{maindir}/sites_with_gymtreepfts.nc')
sites_with_gymtreepfts = sites_with_gymtreepfts['sites_with_gymtreepfts']

In [44]:
maindir = '/glade/work/bbuchovecky/WUE_analysis/sites_from_gridded'
casename = 'clm50_cesm23a02cPPEn08ctsm51d030_1deg_GSWP3V1_hist'
var = 'GSSUNLN'

gss = xr.open_dataset(f'{maindir}/{casename}.clm2.h1.{var}.185001-201412_sites.nc')
gss = gss[var]

In [45]:
# Create slice objects to select the corresponding ang/gym tree PFTs
angtreepft_index_slice = slice(4,9)
gymtreepft_index_slice = slice(1,4)

print(f'angiosperm PFTs:\n{gss.isel(pft=angtreepft_index_slice).pft_name.values}')
print(f'gymnosperm PFTs:\n{gss.isel(pft=gymtreepft_index_slice).pft_name.values}')

angiosperm PFTs:
[b'broadleaf_evergreen_tropical_tree       '
 b'broadleaf_evergreen_temperate_tree      '
 b'broadleaf_deciduous_tropical_tree       '
 b'broadleaf_deciduous_temperate_tree      '
 b'broadleaf_deciduous_boreal_tree         ']
gymnosperm PFTs:
[b'needleleaf_evergreen_temperate_tree     '
 b'needleleaf_evergreen_boreal_tree        '
 b'needleleaf_deciduous_boreal_tree        ']


In [46]:
# Compute the mean across the corresponding ang/gym PFTs
gss_ang = gss.isel(pft=angtreepft_index_slice).mean(dim='pft')
gss_gym = gss.isel(pft=gymtreepft_index_slice).mean(dim='pft')

# Now filter the tree ring sites by ang/gym
gss_ang_site = gss_ang.isel(site=sites_with_angtreepfts)
gss_gym_site = gss_gym.isel(site=sites_with_gymtreepfts)

In [47]:
gss_ang_site

<xarray.DataArray 'GSSUNLN' (site: 125, time: 1980)>
array([[134196.42 , 139721.47 ,  97568.43 , ...,  41491.12 ,  79774.16 ,
         87433.016],
       [198841.78 , 182748.97 , 116590.78 , ..., 107883.78 , 101001.53 ,
        164485.25 ],
       [133413.6  , 128677.41 , 121070.45 , ...,  78022.94 , 131014.68 ,
        135876.69 ],
       ...,
       [248260.64 , 270667.7  , 199105.56 , ...,  82964.555, 167539.39 ,
        156116.28 ],
       [147244.84 , 154375.53 , 141099.72 , ..., 109932.53 , 137357.19 ,
        123790.21 ],
       [147244.84 , 154375.53 , 141099.72 , ..., 109932.53 , 137357.19 ,
        123790.21 ]], dtype=float32)
Coordinates:
    lat      (site) float32 ...
  * time     (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    lon      (site) float32 ...
  * site     (site) int64 5 6 7 8 9 10 11 12 ... 359 361 369 380 386 393 409 410

In [53]:
gss_gym_site

<xarray.DataArray 'GSSUNLN' (site: 265, time: 1980)>
array([[ 34426.383,  21821.295,  25989.549, ...,  91234.07 ,  59329.496,
         30811.541],
       [ 17487.857,  12572.721,  23111.863, ...,  76585.55 ,  47217.71 ,
         22103.652],
       [122220.66 , 107057.84 ,  85915.64 , ...,  22659.848,  46396.47 ,
         73479.62 ],
       ...,
       [ 62248.594,  74356.336,  82490.76 , ...,  49917.887,  53852.176,
         51118.62 ],
       [ 55180.594,  61057.68 ,  93323.7  , ...,  65603.695,  56410.844,
         35428.234],
       [ 18954.303,  16861.727,  70692.3  , ...,  52840.027,  28435.89 ,
         16097.93 ]], dtype=float32)
Coordinates:
    lat      (site) float32 ...
  * time     (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    lon      (site) float32 ...
  * site     (site) int64 0 1 22 23 24 25 26 27 ... 423 424 425 426 429 430 431